# Eligibility : Intubation+Extubation 정보를 합침

## 데이터베이스 연결

In [64]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
import time
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine

In [65]:
#pip freeze > requirements.txt

In [66]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

try:
    tunnel= SSHTunnelForwarder(
         ('', ),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="",
         ssh_password="", 
         remote_bind_address=('localhost',))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': '',
             'password': '',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()



tunnel connected
database connected


In [67]:
intubation1 = pd.read_csv('../outputs/intubation1.csv', parse_dates=['intubationtime'])
extubation1 = pd.read_csv('../outputs/extubation1.csv', parse_dates=['extubationtime'])

In [68]:
intubation1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7724 entries, 0 to 7723
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   subject_id      7724 non-null   int64         
 1   hadm_id         7724 non-null   int64         
 2   stay_id         7724 non-null   int64         
 3   intubationtime  7724 non-null   datetime64[ns]
 4   itemid          7724 non-null   int64         
 5   patientweight   7724 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 362.2 KB


## Intubation_Extubation 정보를 붙여주자.

In [69]:
#intubation 기준으로 파일 붙여주자
intubation_extubation = intubation1 >> left_join(extubation1, by = ("subject_id", "hadm_id"))
intubation_extubation.info()
#extubationtime이랑 intubationtime이 object로 되어있는데 이를 datetime으로 바꿔줘야함

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8948 entries, 0 to 8947
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   subject_id       8948 non-null   int64         
 1   hadm_id          8948 non-null   int64         
 2   stay_id_x        8948 non-null   int64         
 3   intubationtime   8948 non-null   datetime64[ns]
 4   itemid_x         8948 non-null   int64         
 5   patientweight_x  8948 non-null   float64       
 6   stay_id_y        7431 non-null   float64       
 7   extubationtime   7431 non-null   datetime64[ns]
 8   itemid_y         7431 non-null   float64       
 9   patientweight_y  7431 non-null   float64       
 10  extubationcause  7431 non-null   object        
dtypes: datetime64[ns](2), float64(4), int64(4), object(1)
memory usage: 769.1+ KB


In [70]:
#intubation_extubation['intubationtime'] = pd.to_datetime(intubation_extubation['intubationtime'])
#intubation_extubation['extubationtime'] = pd.to_datetime(intubation_extubation['extubationtime'])

In [71]:
#여기서 extubation에서 intubation time을 빼면 몇시간인지 나올텐데 이걸 intubationtoextubationtime으로 컬럼을 생성할 것임 근데 그냥하면 에러남
#이걸로 재삽관까지 걸린 시간을 알 수 있음
intubation_extubation['intubationtoextubationtime'] = intubation_extubation['extubationtime'] - intubation_extubation['intubationtime']

In [72]:
#일단 중복 컬럼명들을 좀 손봐주겠음
intubation_extubation.rename(columns={'itemid_x':'int_itemid', 'itemid_y':'ext_itemid', 'patientweight_x':'int_weight', 'patientweight_y':'ext_weight'}, inplace=True)

In [73]:
intubation_extubation.drop(columns=['stay_id_x', 'stay_id_y'], inplace=True)

In [74]:
intubation_extubation.dtypes #각 컬럼들이 정리 된 모습.

subject_id                              int64
hadm_id                                 int64
intubationtime                 datetime64[ns]
int_itemid                              int64
int_weight                            float64
extubationtime                 datetime64[ns]
ext_itemid                            float64
ext_weight                            float64
extubationcause                        object
intubationtoextubationtime    timedelta64[ns]
dtype: object

In [75]:
intubation_extubation.head(10)

,subject_id,hadm_id,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime
0,10001884,26184834,2131-01-11 04:30:00,224385,65.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,1 days 13:10:00
1,10003400,23559586,2137-08-17 21:21:00,224385,90.5,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,3 days 18:19:00
2,10004401,27939719,2144-04-11 05:03:00,224385,120.0,NaT,NaN,NaN,NaN,NaT
3,10004401,29988601,2144-01-27 19:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2 days 17:30:00
4,10004401,29988601,2144-01-30 13:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,-1 days +23:30:00
5,10005817,28661809,2135-01-06 08:33:00,224385,88.8,2135-01-19 14:00:00,227194.0,88.8,Planned Extubation,13 days 05:27:00
6,10007818,22987108,2146-06-22 12:49:00,224385,86.2,NaT,NaN,NaN,NaN,NaT
7,10010058,26359957,2147-11-18 02:35:00,224385,60.0,NaT,NaN,NaN,NaN,NaT
8,10011365,26712576,2166-02-14 17:59:00,224385,46.3,2166-02-14 09:48:00,227194.0,46.3,Planned Extubation,-1 days +15:49:00
9,10015931,22130791,2177-03-25 09:57:00,224385,59.5,2177-03-29 13:11:00,227194.0,59.5,Planned Extubation,4 days 03:14:00


In [76]:
#일단 extubationtime에 isna가 있는지 확인?
intubation_extubation['extubationtime'].isna().sum() #없음

1517

In [77]:
#이 데이터들은 언제 발관을 했는지 시점을 확인할 수 없는데, csv로 저장하고 퇴원 데이터를 붙여서 확인해봐야할 것임
check_extubation1 = intubation_extubation[intubation_extubation['intubationtoextubationtime'].isna()]
check_extubation1.to_csv('../outputs/check_extubation1.csv', index=False)


In [78]:
curs.execute("select * from mimiciv_hosp.admissions;")
columns_name = [desc[0] for desc in curs.description]
admissions = pd.DataFrame(curs.fetchall(), columns=columns_name) 
admissions

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaT,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaT,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaT,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaT,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaT,EU OBSERVATION,P51VDL,EMERGENCY ROOM,None,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,19999828,25744818,2149-01-08 16:44:00,2149-01-18 17:00:00,NaT,EW EMER.,P75BG6,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2149-01-08 09:11:00,2149-01-08 18:12:00,0
431227,19999828,29734428,2147-07-18 16:23:00,2147-08-04 18:10:00,NaT,EW EMER.,P16C7J,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,2147-07-17 17:18:00,2147-07-18 17:34:00,0
431228,19999840,21033226,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,EW EMER.,P58A9J,EMERGENCY ROOM,DIED,Other,ENGLISH,WIDOWED,WHITE,2164-09-10 11:09:00,2164-09-10 14:46:00,1
431229,19999840,26071774,2164-07-25 00:27:00,2164-07-28 12:15:00,NaT,EW EMER.,P506DE,EMERGENCY ROOM,HOME,Other,ENGLISH,WIDOWED,WHITE,2164-07-24 21:16:00,2164-07-25 01:20:00,0


In [79]:
check_extubation2 = check_extubation1 >> left_join(admissions, by = ("subject_id", "hadm_id")) >> select("subject_id", "hadm_id", "admittime", "intubationtime","int_itemid","int_weight","extubationtime","ext_itemid",'ext_weight',"extubationcause","intubationtoextubationtime","dischtime", "deathtime")
check_extubation2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1517 entries, 0 to 1516
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   subject_id                  1517 non-null   int64          
 1   hadm_id                     1517 non-null   int64          
 2   admittime                   1517 non-null   datetime64[ns] 
 3   intubationtime              1517 non-null   datetime64[ns] 
 4   int_itemid                  1517 non-null   int64          
 5   int_weight                  1517 non-null   float64        
 6   extubationtime              0 non-null      datetime64[ns] 
 7   ext_itemid                  0 non-null      float64        
 8   ext_weight                  0 non-null      float64        
 9   extubationcause             0 non-null      object         
 10  intubationtoextubationtime  0 non-null      timedelta64[ns]
 11  dischtime                   1517 non-null  

In [80]:
check_extubation2
#데이터를 보면, 사망일과 퇴원일이 같은경우/다른경우가 있고, 사망은 없고 퇴원한 경우가 있음. 사망한 경우 사망하면 기기를 제거하므로 deathtime 을 쓰는 것이 적절할 것임. discharge를 할 경우 관을 유지하나? 보통 발관할 것 같은데..? 그래도 해당 병원에
#이 경우, extubationcause 에 deathtime 으로 적어서 deathtime 을 사용했음을 보여주려고 함.
#사망은 869건이니까 1840-869 = 971건이 퇴원한 경우임. 이 경우는 discharge time을 사용할지? 흠 역시 dischargetime 을 사용하는것이 적절할 것같음, 이 경우에도 extubationcause에 dischargetime 으로 적어야 할 것 같음
#그럼 이경우 코드를 어떻게 짤 수 있냐? 데이터 둘다 있으면 deathtime 에게 우선 순위를 주겠음(사유, 보통 더 정확함)

,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime
0,10004401,27939719,2144-04-11 03:31:00,2144-04-11 05:03:00,224385,120.0,NaT,NaN,NaN,NaN,NaT,2144-04-13 17:31:00,NaT
1,10007818,22987108,2146-06-10 16:37:00,2146-06-22 12:49:00,224385,86.2,NaT,NaN,NaN,NaN,NaT,2146-07-12 00:00:00,2146-07-12 20:50:00
2,10010058,26359957,2147-11-18 02:34:00,2147-11-18 02:35:00,224385,60.0,NaT,NaN,NaN,NaN,NaT,2147-11-19 04:00:00,2147-11-19 04:00:00
3,10017531,22580355,2159-09-22 19:30:00,2159-09-23 16:01:00,224385,119.0,NaT,NaN,NaN,NaN,NaT,2159-10-24 13:40:00,NaT
4,10019957,28761725,2135-11-11 10:45:00,2135-11-11 14:35:00,224385,76.6,NaT,NaN,NaN,NaN,NaT,2135-11-15 18:00:00,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1512,19875779,26894843,2162-06-24 19:12:00,2162-06-24 19:38:00,224385,154.7,NaT,NaN,NaN,NaN,NaT,2162-07-17 12:45:00,NaT
1513,19929286,22584344,2192-12-05 02:06:00,2192-12-05 09:00:00,224385,70.5,NaT,NaN,NaN,NaN,NaT,2193-01-10 19:30:00,NaT
1514,19949739,25237531,2145-05-31 14:41:00,2145-06-03 04:00:00,224385,57.5,NaT,NaN,NaN,NaN,NaT,2145-06-03 05:16:00,2145-06-03 05:16:00
1515,19960105,27292691,2147-08-06 18:06:00,2147-08-15 12:40:00,224385,56.8,NaT,NaN,NaN,NaN,NaT,2147-08-23 14:16:00,NaT


In [81]:
##사망일 채워 넣기 전 파일
intubation_extubation.to_csv('../outputs/intubation_extubation.csv', index=False)

In [82]:
#hadm_id 별로 intubationtime 이 총 몇개 있는지 확인하고 싶음(각 입원에서 몇번 intubation을 했는지 확인하고 싶음) 이걸 intcount 라는 컬럼에 넣어주면 좋겠음
intubation_extubation['intcount'] = intubation_extubation.groupby('hadm_id')['intubationtime'].transform('count')
#intseq 라는 컬럼에 같은 hadm_id 에서 intubationtime 이 빠른 순서대로 1, 2 ... 와 같이 넘버링하고 싶어
intubation_extubation['intseq'] = intubation_extubation.groupby('hadm_id').cumcount() + 1

In [83]:
intubation_extubation2 = intubation_extubation >> left_join(admissions, by = ("subject_id", "hadm_id")) >> select("subject_id", "hadm_id", "admittime","intubationtime","int_itemid","int_weight","extubationtime","ext_itemid",'ext_weight',"extubationcause","intubationtoextubationtime","dischtime", "deathtime",'intcount','intseq')
intubation_extubation2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8948 entries, 0 to 8947
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   subject_id                  8948 non-null   int64          
 1   hadm_id                     8948 non-null   int64          
 2   admittime                   8948 non-null   datetime64[ns] 
 3   intubationtime              8948 non-null   datetime64[ns] 
 4   int_itemid                  8948 non-null   int64          
 5   int_weight                  8948 non-null   float64        
 6   extubationtime              7431 non-null   datetime64[ns] 
 7   ext_itemid                  7431 non-null   float64        
 8   ext_weight                  7431 non-null   float64        
 9   extubationcause             7431 non-null   object         
 10  intubationtoextubationtime  7431 non-null   timedelta64[ns]
 11  dischtime                   8948 non-null  

In [84]:
# intcount가 1인 경우, 다음 로직을 적용하여 채우도록하기
# #deathtime과 dischtime 둘 다 있는 경우, deathtime을 사용하여 extubationtime을 업데이트하고 extubationcause에 "deathtime" 문자열을 입력합니다.
# #deathtime이 있는 경우 (이전의 로직과 겹치지 않는다는 가정하에) 역시 deathtime을 사용하여 extubationtime을 업데이트하고 extubationcause에 "deathtime" 문자열을 입력합니다.
# #deathtime이 없고 dischtime만 있는 경우, dischtime을 사용하여 extubationtime을 업데이트하고 extubationcause에 "dischargetime" 문자열을 입력합니다.
# # deathtime과 dischtime 둘 다 있는 경우
# mask_both = (~check_extubation2['deathtime'].isna()) & (~check_extubation2['dischtime'].isna())
# check_extubation2.loc[mask_both, 'extubationtime'] = check_extubation2.loc[mask_both, 'deathtime']
# check_extubation2.loc[mask_both, 'extubationcause'] = 'deathtime'

# # deathtime만 있는 경우
# mask_death = (~check_extubation2['deathtime'].isna()) & (check_extubation2['dischtime'].isna())
# check_extubation2.loc[mask_death, 'extubationtime'] = check_extubation2.loc[mask_death, 'deathtime']
# check_extubation2.loc[mask_death, 'extubationcause'] = 'deathtime'

# # dischtime만 있는 경우
# mask_discharge = (check_extubation2['deathtime'].isna()) & (~check_extubation2['dischtime'].isna())
# check_extubation2.loc[mask_discharge, 'extubationtime'] = check_extubation2.loc[mask_discharge, 'dischtime']
# check_extubation2.loc[mask_discharge, 'extubationcause'] = 'dischargetime'


df = intubation_extubation2

def replace_extubationtime_na(df):
    # intcount가 1이고 extubationtime이 NaT인 행만 선택
    mask = (df['intcount'] == 1) & (df['extubationtime'].isna())
    
    # deathtime을 사용하여 extubationtime의 NaT 값을 대체
    condition_death = mask & (~df['deathtime'].isna())
    df.loc[condition_death, 'extubationtime'] = df.loc[condition_death, 'deathtime']
    df.loc[condition_death, 'extubationcause'] = 'deathtime'

    # dischtime을 사용하여 extubationtime의 NaT 값을 대체
    condition_discharge = mask & (~df['dischtime'].isna())
    df.loc[condition_discharge, 'extubationtime'] = df.loc[condition_discharge, 'dischtime']
    df.loc[condition_discharge, 'extubationcause'] = 'dischtime'

    return df

# 데이터프레임에 함수 적용
df = replace_extubationtime_na(df)



In [85]:
df

,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime,intcount,intseq
0,10001884,26184834,2131-01-07 20:39:00,2131-01-11 04:30:00,224385,65.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,1 days 13:10:00,2131-01-20 05:15:00,2131-01-20 05:15:00,1,1
1,10003400,23559586,2137-08-04 00:07:00,2137-08-17 21:21:00,224385,90.5,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,3 days 18:19:00,2137-09-02 17:05:00,2137-09-02 17:05:00,1,1
2,10004401,27939719,2144-04-11 03:31:00,2144-04-11 05:03:00,224385,120.0,2144-04-13 17:31:00,NaN,NaN,dischtime,NaT,2144-04-13 17:31:00,NaT,1,1
3,10004401,29988601,2144-01-23 07:58:00,2144-01-27 19:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2 days 17:30:00,2144-02-06 11:45:00,NaT,2,1
4,10004401,29988601,2144-01-23 07:58:00,2144-01-30 13:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,-1 days +23:30:00,2144-02-06 11:45:00,NaT,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8943,19983257,21588174,2166-06-13 14:21:00,2166-06-14 08:58:00,224385,118.6,2166-06-14 10:47:00,227194.0,118.6,Planned Extubation,0 days 01:49:00,2166-06-19 21:15:00,2166-06-19 21:15:00,2,1
8944,19983257,21588174,2166-06-13 14:21:00,2166-06-19 09:20:00,224385,118.6,2166-06-14 10:47:00,227194.0,118.6,Planned Extubation,-5 days +01:27:00,2166-06-19 21:15:00,2166-06-19 21:15:00,2,2
8945,19984259,20109446,2153-11-29 23:43:00,2153-12-01 11:49:00,224385,72.0,2153-12-05 13:10:00,227194.0,72.0,Planned Extubation,4 days 01:21:00,2153-12-08 17:05:00,NaT,1,1
8946,19985545,21516111,2144-07-22 00:25:00,2144-08-03 14:15:00,224385,70.0,2144-08-04 13:50:00,227194.0,70.0,Planned Extubation,0 days 23:35:00,2144-08-25 16:40:00,NaT,1,1


In [86]:
# #deathtime과 dischtime 둘 다 있는 경우, deathtime을 사용하여 extubationtime을 업데이트하고 extubationcause에 "deathtime" 문자열을 입력합니다.
# #deathtime이 있는 경우 (이전의 로직과 겹치지 않는다는 가정하에) 역시 deathtime을 사용하여 extubationtime을 업데이트하고 extubationcause에 "deathtime" 문자열을 입력합니다.
# #deathtime이 없고 dischtime만 있는 경우, dischtime을 사용하여 extubationtime을 업데이트하고 extubationcause에 "dischargetime" 문자열을 입력합니다.
# # deathtime과 dischtime 둘 다 있는 경우
# mask_both = (~check_extubation2['deathtime'].isna()) & (~check_extubation2['dischtime'].isna())
# check_extubation2.loc[mask_both, 'extubationtime'] = check_extubation2.loc[mask_both, 'deathtime']
# check_extubation2.loc[mask_both, 'extubationcause'] = 'deathtime'

# # deathtime만 있는 경우
# mask_death = (~check_extubation2['deathtime'].isna()) & (check_extubation2['dischtime'].isna())
# check_extubation2.loc[mask_death, 'extubationtime'] = check_extubation2.loc[mask_death, 'deathtime']
# check_extubation2.loc[mask_death, 'extubationcause'] = 'deathtime'

# # dischtime만 있는 경우
# mask_discharge = (check_extubation2['deathtime'].isna()) & (~check_extubation2['dischtime'].isna())
# check_extubation2.loc[mask_discharge, 'extubationtime'] = check_extubation2.loc[mask_discharge, 'dischtime']
# check_extubation2.loc[mask_discharge, 'extubationcause'] = 'dischargetime'


In [87]:
#이 로직을  intubation_extubation 원코드에 적용하겠음

In [88]:
# intubation_extubation2 = intubation_extubation >> left_join(admissions, by = ("subject_id", "hadm_id")) >> select("subject_id", "hadm_id", "admittime","intubationtime","int_itemid","int_weight","extubationtime","ext_itemid",'ext_weight',"extubationcause","intubationtoextubationtime","dischtime", "deathtime")
# intubation_extubation2.info()

In [89]:
# #주의! extubationtime이 이미 데이터가 있는 경우 replace하면 안됨!
# #deathtime과 dischtime 둘 다 있는 경우, deathtime을 사용하여 extubationtime을 업데이트하고 extubationcause에 "deathtime" 문자열을 입력합니다.
# #deathtime이 있는 경우 (이전의 로직과 겹치지 않는다는 가정하에) 역시 deathtime을 사용하여 extubationtime을 업데이트하고 extubationcause에 "deathtime" 문자열을 입력합니다.
# #deathtime이 없고 dischtime만 있는 경우, dischtime을 사용하여 extubationtime을 업데이트하고 extubationcause에 "dischargetime" 문자열을 입력합니다.
# # deathtime과 dischtime 둘 다 있는 경우이면서 extubationtime이 NaN인 경우
# mask_both = (~intubation_extubation2['deathtime'].isna()) & (~intubation_extubation2['dischtime'].isna()) & (intubation_extubation2['extubationtime'].isna())
# intubation_extubation2.loc[mask_both, 'extubationtime'] = intubation_extubation2.loc[mask_both, 'deathtime']
# intubation_extubation2.loc[mask_both, 'extubationcause'] = 'deathtime'
# #
# # deathtime만 있는 경우이면서 extubationtime이 NaN인 경우
# mask_death = (~intubation_extubation2['deathtime'].isna()) & (intubation_extubation2['dischtime'].isna()) & (intubation_extubation2['extubationtime'].isna())
# intubation_extubation2.loc[mask_death, 'extubationtime'] = intubation_extubation2.loc[mask_death, 'deathtime']
# intubation_extubation2.loc[mask_death, 'extubationcause'] = 'deathtime'

# # dischtime만 있는 경우이면서 extubationtime이 NaN인 경우
# mask_discharge = (intubation_extubation2['deathtime'].isna()) & (~intubation_extubation2['dischtime'].isna()) & (intubation_extubation2['extubationtime'].isna())
# intubation_extubation2.loc[mask_discharge, 'extubationtime'] = intubation_extubation2.loc[mask_discharge, 'dischtime']
# intubation_extubation2.loc[mask_discharge, 'extubationcause'] = 'dischargetime'


In [90]:
df['extubationcause'].value_counts()

extubationcause
Planned Extubation                              7012
dischtime                                       1456
Unplanned Extuabtion (patient-initiated)         388
Unplanned Extuabtion (non-patient initiated)      31
Name: count, dtype: int64

intubation_extubation2 = df

In [91]:
intubation_extubation2 = df

In [92]:
#intubation에서 extubation까지 수행한 시간을 계산하겠음 (ventilator 적용시간 보고자함)
intubation_extubation2['intubationtoextubationtime'] = intubation_extubation2['extubationtime'] - intubation_extubation2['intubationtime']

In [93]:
intubation_extubation2

,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime,intcount,intseq
0,10001884,26184834,2131-01-07 20:39:00,2131-01-11 04:30:00,224385,65.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,1 days 13:10:00,2131-01-20 05:15:00,2131-01-20 05:15:00,1,1
1,10003400,23559586,2137-08-04 00:07:00,2137-08-17 21:21:00,224385,90.5,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,3 days 18:19:00,2137-09-02 17:05:00,2137-09-02 17:05:00,1,1
2,10004401,27939719,2144-04-11 03:31:00,2144-04-11 05:03:00,224385,120.0,2144-04-13 17:31:00,NaN,NaN,dischtime,2 days 12:28:00,2144-04-13 17:31:00,NaT,1,1
3,10004401,29988601,2144-01-23 07:58:00,2144-01-27 19:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2 days 17:30:00,2144-02-06 11:45:00,NaT,2,1
4,10004401,29988601,2144-01-23 07:58:00,2144-01-30 13:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,-1 days +23:30:00,2144-02-06 11:45:00,NaT,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8943,19983257,21588174,2166-06-13 14:21:00,2166-06-14 08:58:00,224385,118.6,2166-06-14 10:47:00,227194.0,118.6,Planned Extubation,0 days 01:49:00,2166-06-19 21:15:00,2166-06-19 21:15:00,2,1
8944,19983257,21588174,2166-06-13 14:21:00,2166-06-19 09:20:00,224385,118.6,2166-06-14 10:47:00,227194.0,118.6,Planned Extubation,-5 days +01:27:00,2166-06-19 21:15:00,2166-06-19 21:15:00,2,2
8945,19984259,20109446,2153-11-29 23:43:00,2153-12-01 11:49:00,224385,72.0,2153-12-05 13:10:00,227194.0,72.0,Planned Extubation,4 days 01:21:00,2153-12-08 17:05:00,NaT,1,1
8946,19985545,21516111,2144-07-22 00:25:00,2144-08-03 14:15:00,224385,70.0,2144-08-04 13:50:00,227194.0,70.0,Planned Extubation,0 days 23:35:00,2144-08-25 16:40:00,NaT,1,1


In [94]:
intubation_extubation2[intubation_extubation2['intubationtoextubationtime'].isna()]

,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime,intcount,intseq
108,10136619,23362878,2141-01-16 16:18:00,2141-01-21 07:55:00,224385,55.5,NaT,NaN,NaN,NaN,NaT,2141-01-29 11:00:00,2141-01-29 11:00:00,2,1
109,10136619,23362878,2141-01-16 16:18:00,2141-01-23 04:30:00,224385,55.5,NaT,NaN,NaN,NaN,NaT,2141-01-29 11:00:00,2141-01-29 11:00:00,2,2
126,10151556,20815067,2184-02-07 23:34:00,2184-02-14 16:19:00,224385,95.0,NaT,NaN,NaN,NaN,NaT,2184-03-31 19:00:00,NaT,2,1
127,10151556,20815067,2184-02-07 23:34:00,2184-03-21 00:25:00,224385,95.0,NaT,NaN,NaN,NaN,NaT,2184-03-31 19:00:00,NaT,2,2
1077,11117353,20679385,2124-12-31 05:13:00,2124-12-31 12:20:00,224385,85.6,NaT,NaN,NaN,NaN,NaT,2125-01-11 14:00:00,NaT,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8752,19733670,25713858,2183-06-15 04:32:00,2183-06-15 13:15:00,224385,58.8,NaT,NaN,NaN,NaN,NaT,2183-07-02 19:10:00,2183-07-02 19:10:00,2,1
8879,19733670,25713858,2183-06-15 04:32:00,2183-06-28 17:30:00,224385,58.8,NaT,NaN,NaN,NaN,NaT,2183-07-02 19:10:00,2183-07-02 19:10:00,2,2
8904,19789144,23993014,2152-10-26 02:45:00,2152-11-11 22:59:00,224385,51.5,NaT,NaN,NaN,NaN,NaT,2152-11-17 00:30:00,2152-11-17 00:30:00,2,1
8905,19789144,23993014,2152-10-26 02:45:00,2152-11-15 14:30:00,224385,51.5,NaT,NaN,NaN,NaN,NaT,2152-11-17 00:30:00,2152-11-17 00:30:00,2,2


In [95]:
b1 = intubation_extubation2[intubation_extubation2['intubationtoextubationtime'].isna()]

In [96]:
recheck_exttime_hadm = b1['hadm_id']
recheck_exttime_hadm.drop_duplicates(inplace=True)
recheck_exttime_hadm

108     23362878
126     20815067
1077    20679385
1200    22946206
1319    20635802
1441    26321077
1450    20986748
1520    28893487
2016    25055035
2240    21903132
2331    28540703
2854    20297853
3114    20070255
5363    27238072
5512    24817331
5544    26853910
6067    26300037
6175    21582058
6237    23237338
6622    22720579
6752    27093340
7002    25704244
7332    26731051
7571    28849038
8177    24357541
8384    29034642
8658    23981090
8722    25237531
8752    25713858
8904    23993014
Name: hadm_id, dtype: int64

In [97]:
extubation_all = pd.read_csv('../outputs/extubation_all.csv')
extubation_all.head(2)

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10001884,26184834,37510196,35966,2131-01-12 17:40:00,2131-01-12 17:41:00,2131-01-12 17:50:00,227194,1.0,NaN,...,6470885,6470885,Intubation/Extubation,Task,65.0,0,0,FinishedRunning,1.0,0.0
1,10002013,23581541,39060235,65197,2160-05-18 18:00:00,2160-05-18 18:01:00,2160-05-18 17:59:00,227194,1.0,NaN,...,3546453,3546453,Intubation/Extubation,Task,96.0,0,0,FinishedRunning,1.0,0.0


In [98]:
extubation_all[extubation_all['hadm_id'].isin(recheck_exttime_hadm)] #일치하는 정보가 아예 없는 것을 알 수 있다. 

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,orderid,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate


In [99]:
b1

,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime,intcount,intseq
108,10136619,23362878,2141-01-16 16:18:00,2141-01-21 07:55:00,224385,55.5,NaT,NaN,NaN,NaN,NaT,2141-01-29 11:00:00,2141-01-29 11:00:00,2,1
109,10136619,23362878,2141-01-16 16:18:00,2141-01-23 04:30:00,224385,55.5,NaT,NaN,NaN,NaN,NaT,2141-01-29 11:00:00,2141-01-29 11:00:00,2,2
126,10151556,20815067,2184-02-07 23:34:00,2184-02-14 16:19:00,224385,95.0,NaT,NaN,NaN,NaN,NaT,2184-03-31 19:00:00,NaT,2,1
127,10151556,20815067,2184-02-07 23:34:00,2184-03-21 00:25:00,224385,95.0,NaT,NaN,NaN,NaN,NaT,2184-03-31 19:00:00,NaT,2,2
1077,11117353,20679385,2124-12-31 05:13:00,2124-12-31 12:20:00,224385,85.6,NaT,NaN,NaN,NaN,NaT,2125-01-11 14:00:00,NaT,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8752,19733670,25713858,2183-06-15 04:32:00,2183-06-15 13:15:00,224385,58.8,NaT,NaN,NaN,NaN,NaT,2183-07-02 19:10:00,2183-07-02 19:10:00,2,1
8879,19733670,25713858,2183-06-15 04:32:00,2183-06-28 17:30:00,224385,58.8,NaT,NaN,NaN,NaN,NaT,2183-07-02 19:10:00,2183-07-02 19:10:00,2,2
8904,19789144,23993014,2152-10-26 02:45:00,2152-11-11 22:59:00,224385,51.5,NaT,NaN,NaN,NaN,NaT,2152-11-17 00:30:00,2152-11-17 00:30:00,2,1
8905,19789144,23993014,2152-10-26 02:45:00,2152-11-15 14:30:00,224385,51.5,NaT,NaN,NaN,NaN,NaT,2152-11-17 00:30:00,2152-11-17 00:30:00,2,2


In [100]:
#여기서 intcount는 해당 입원에 총 몇번의 intubation을 했는지를 의미한다.
#그러므로 같은 hadm_id에서 intcount와 seq가 같다면, dichtime 또는 deathtime 으로 extubationtime을 대체할 수 있다.
#그외는 마지막 seq에 도달하기 전이므로, 다음 seq의 intubationtime 을 이전 seq의 extubationtime으로 대체한다.

intubation_extubation2['intubationtime'] = pd.to_datetime(intubation_extubation2['intubationtime'])

In [101]:
intubation_extubation2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8948 entries, 0 to 8947
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype          
---  ------                      --------------  -----          
 0   subject_id                  8948 non-null   int64          
 1   hadm_id                     8948 non-null   int64          
 2   admittime                   8948 non-null   datetime64[ns] 
 3   intubationtime              8948 non-null   datetime64[ns] 
 4   int_itemid                  8948 non-null   int64          
 5   int_weight                  8948 non-null   float64        
 6   extubationtime              8887 non-null   datetime64[ns] 
 7   ext_itemid                  7431 non-null   float64        
 8   ext_weight                  7431 non-null   float64        
 9   extubationcause             8887 non-null   object         
 10  intubationtoextubationtime  8887 non-null   timedelta64[ns]
 11  dischtime                   8948 non-null  

In [102]:
intubation_extubation3 = intubation_extubation2

In [103]:
intubation_extubation3 = intubation_extubation3.sort_values(by=['subject_id','hadm_id','intubationtime','intseq'])

In [104]:
intubation_extubation3

,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime,intcount,intseq
0,10001884,26184834,2131-01-07 20:39:00,2131-01-11 04:30:00,224385,65.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,1 days 13:10:00,2131-01-20 05:15:00,2131-01-20 05:15:00,1,1
1,10003400,23559586,2137-08-04 00:07:00,2137-08-17 21:21:00,224385,90.5,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,3 days 18:19:00,2137-09-02 17:05:00,2137-09-02 17:05:00,1,1
2,10004401,27939719,2144-04-11 03:31:00,2144-04-11 05:03:00,224385,120.0,2144-04-13 17:31:00,NaN,NaN,dischtime,2 days 12:28:00,2144-04-13 17:31:00,NaT,1,1
3,10004401,29988601,2144-01-23 07:58:00,2144-01-27 19:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2 days 17:30:00,2144-02-06 11:45:00,NaT,2,1
4,10004401,29988601,2144-01-23 07:58:00,2144-01-30 13:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,-1 days +23:30:00,2144-02-06 11:45:00,NaT,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8861,19997367,20617667,2126-04-20 07:15:00,2126-05-05 18:20:00,224385,59.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2 days 23:09:00,2126-05-19 14:15:00,NaT,6,5
8862,19997367,20617667,2126-04-20 07:15:00,2126-05-05 18:20:00,224385,59.0,2126-04-22 08:23:00,227194.0,59.0,Planned Extubation,-14 days +14:03:00,2126-05-19 14:15:00,NaT,6,6
8863,19999068,21606769,2161-08-24 04:10:00,2161-08-25 15:34:00,224385,55.8,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2 days 22:01:00,2161-09-02 19:00:00,NaT,1,1
8836,19999442,26785317,2148-11-19 10:00:00,2148-11-19 19:00:00,224385,107.5,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,0 days 19:15:00,2148-12-04 16:25:00,NaT,1,1


In [105]:

#여기서 intcount는 해당 입원에 총 몇번의 intubation을 했는지를 의미한다.
#그러므로 같은 hadm_id에서 intcount와 seq가 같다면, dichtime 또는 deathtime 으로 extubationtime을 대체할 수 있다.
#그외는 마지막 seq에 도달하기 전이므로, 다음 seq의 intubationtime 을 이전 seq의 extubationtime으로 대체한다.

df = intubation_extubation3


def replace_extubationtime_non(df):
    # intcount와 intseq가 같고 extubationtime이 NaT인 행만 선택
    mask_last_seq = (df['intcount'] == df['intseq']) & df['extubationtime'].isna()
    
    # deathtime 또는 dischtime으로 extubationtime의 NaT 값을 대체
    df.loc[mask_last_seq, 'extubationtime'] = df.loc[mask_last_seq, 'dischtime'].fillna(df.loc[mask_last_seq, 'deathtime'])
    df.loc[mask_last_seq, 'extubationcause'] = 'dischtime'
    
    # 그 외의 경우: 다음 seq의 intubationtime을 이전 seq의 extubationtime으로 대체
    for idx, row in df.iterrows():
        if row['intseq'] < row['intcount'] and pd.isna(row['extubationtime']):
            next_row = df[(df['hadm_id'] == row['hadm_id']) & (df['intseq'] == row['intseq'] + 1)]
            if not next_row.empty:
                df.at[idx, 'extubationtime'] = next_row['intubationtime'].values[0]
                df.at[idx, 'extubationcause'] = 'next_intubationtime'
    
    return df

# 데이터프레임에 함수 적용
df = replace_extubationtime_non(df)


In [106]:
df

,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime,intcount,intseq
0,10001884,26184834,2131-01-07 20:39:00,2131-01-11 04:30:00,224385,65.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,1 days 13:10:00,2131-01-20 05:15:00,2131-01-20 05:15:00,1,1
1,10003400,23559586,2137-08-04 00:07:00,2137-08-17 21:21:00,224385,90.5,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,3 days 18:19:00,2137-09-02 17:05:00,2137-09-02 17:05:00,1,1
2,10004401,27939719,2144-04-11 03:31:00,2144-04-11 05:03:00,224385,120.0,2144-04-13 17:31:00,NaN,NaN,dischtime,2 days 12:28:00,2144-04-13 17:31:00,NaT,1,1
3,10004401,29988601,2144-01-23 07:58:00,2144-01-27 19:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2 days 17:30:00,2144-02-06 11:45:00,NaT,2,1
4,10004401,29988601,2144-01-23 07:58:00,2144-01-30 13:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,-1 days +23:30:00,2144-02-06 11:45:00,NaT,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8861,19997367,20617667,2126-04-20 07:15:00,2126-05-05 18:20:00,224385,59.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2 days 23:09:00,2126-05-19 14:15:00,NaT,6,5
8862,19997367,20617667,2126-04-20 07:15:00,2126-05-05 18:20:00,224385,59.0,2126-04-22 08:23:00,227194.0,59.0,Planned Extubation,-14 days +14:03:00,2126-05-19 14:15:00,NaT,6,6
8863,19999068,21606769,2161-08-24 04:10:00,2161-08-25 15:34:00,224385,55.8,2161-08-28 13:35:00,227194.0,55.8,Planned Extubation,2 days 22:01:00,2161-09-02 19:00:00,NaT,1,1
8836,19999442,26785317,2148-11-19 10:00:00,2148-11-19 19:00:00,224385,107.5,2148-11-20 14:15:00,227194.0,107.5,Planned Extubation,0 days 19:15:00,2148-12-04 16:25:00,NaT,1,1


In [107]:
df[df['extubationtime'].isna()]

,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime,intcount,intseq


In [108]:
intubation_extubation3 = df

In [109]:
#intubation에서 extubation까지 수행한 시간을 계산하겠음 (ventilator 적용시간 보고자함)
intubation_extubation3['intubationtoextubationtime'] = intubation_extubation3['extubationtime'] - intubation_extubation3['intubationtime']

In [110]:
intubation_extubation3.head(50)


,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime,intcount,intseq
0,10001884,26184834,2131-01-07 20:39:00,2131-01-11 04:30:00,224385,65.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,1 days 13:10:00,2131-01-20 05:15:00,2131-01-20 05:15:00,1,1
1,10003400,23559586,2137-08-04 00:07:00,2137-08-17 21:21:00,224385,90.5,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,3 days 18:19:00,2137-09-02 17:05:00,2137-09-02 17:05:00,1,1
2,10004401,27939719,2144-04-11 03:31:00,2144-04-11 05:03:00,224385,120.0,2144-04-13 17:31:00,NaN,NaN,dischtime,2 days 12:28:00,2144-04-13 17:31:00,NaT,1,1
3,10004401,29988601,2144-01-23 07:58:00,2144-01-27 19:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2 days 17:30:00,2144-02-06 11:45:00,NaT,2,1
4,10004401,29988601,2144-01-23 07:58:00,2144-01-30 13:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,-1 days +23:30:00,2144-02-06 11:45:00,NaT,2,2
5,10005817,28661809,2135-01-03 21:54:00,2135-01-06 08:33:00,224385,88.8,2135-01-19 14:00:00,227194.0,88.8,Planned Extubation,13 days 05:27:00,2135-01-19 18:36:00,2135-01-19 18:36:00,1,1
6,10007818,22987108,2146-06-10 16:37:00,2146-06-22 12:49:00,224385,86.2,2146-07-12 00:00:00,NaN,NaN,dischtime,19 days 11:11:00,2146-07-12 00:00:00,2146-07-12 20:50:00,1,1
7,10010058,26359957,2147-11-18 02:34:00,2147-11-18 02:35:00,224385,60.0,2147-11-19 04:00:00,NaN,NaN,dischtime,1 days 01:25:00,2147-11-19 04:00:00,2147-11-19 04:00:00,1,1
8,10011365,26712576,2166-02-11 14:09:00,2166-02-14 17:59:00,224385,46.3,2166-02-14 09:48:00,227194.0,46.3,Planned Extubation,-1 days +15:49:00,2166-02-20 00:20:00,2166-02-20 00:20:00,1,1
9,10015931,22130791,2177-03-24 21:47:00,2177-03-25 09:57:00,224385,59.5,2177-03-29 13:11:00,227194.0,59.5,Planned Extubation,4 days 03:14:00,2177-03-29 14:15:00,2177-03-29 14:15:00,1,1


In [111]:
#만약 intubationtoextubationtime 이 음수이면서, intcount과 intseq가 같다면 dischtime 으로 extubationtime을 대체할 수 있다.
# intubationtoextubationtime이 음수인 경우 및 intcount와 intseq가 같은 경우, extubationtime을 dischtime으로 대체
mask_negative_time = (intubation_extubation3['intubationtoextubationtime'] < pd.Timedelta(0)) & (intubation_extubation3['intcount'] == intubation_extubation3['intseq'])
intubation_extubation3.loc[mask_negative_time, 'extubationtime'] = intubation_extubation3.loc[mask_negative_time, 'dischtime']

In [112]:
intubation_extubation3.head(30)

,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime,intcount,intseq
0,10001884,26184834,2131-01-07 20:39:00,2131-01-11 04:30:00,224385,65.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,1 days 13:10:00,2131-01-20 05:15:00,2131-01-20 05:15:00,1,1
1,10003400,23559586,2137-08-04 00:07:00,2137-08-17 21:21:00,224385,90.5,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,3 days 18:19:00,2137-09-02 17:05:00,2137-09-02 17:05:00,1,1
2,10004401,27939719,2144-04-11 03:31:00,2144-04-11 05:03:00,224385,120.0,2144-04-13 17:31:00,NaN,NaN,dischtime,2 days 12:28:00,2144-04-13 17:31:00,NaT,1,1
3,10004401,29988601,2144-01-23 07:58:00,2144-01-27 19:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2 days 17:30:00,2144-02-06 11:45:00,NaT,2,1
4,10004401,29988601,2144-01-23 07:58:00,2144-01-30 13:00:00,224385,76.0,2144-02-06 11:45:00,227194.0,76.0,Planned Extubation,-1 days +23:30:00,2144-02-06 11:45:00,NaT,2,2
5,10005817,28661809,2135-01-03 21:54:00,2135-01-06 08:33:00,224385,88.8,2135-01-19 14:00:00,227194.0,88.8,Planned Extubation,13 days 05:27:00,2135-01-19 18:36:00,2135-01-19 18:36:00,1,1
6,10007818,22987108,2146-06-10 16:37:00,2146-06-22 12:49:00,224385,86.2,2146-07-12 00:00:00,NaN,NaN,dischtime,19 days 11:11:00,2146-07-12 00:00:00,2146-07-12 20:50:00,1,1
7,10010058,26359957,2147-11-18 02:34:00,2147-11-18 02:35:00,224385,60.0,2147-11-19 04:00:00,NaN,NaN,dischtime,1 days 01:25:00,2147-11-19 04:00:00,2147-11-19 04:00:00,1,1
8,10011365,26712576,2166-02-11 14:09:00,2166-02-14 17:59:00,224385,46.3,2166-02-20 00:20:00,227194.0,46.3,Planned Extubation,-1 days +15:49:00,2166-02-20 00:20:00,2166-02-20 00:20:00,1,1
9,10015931,22130791,2177-03-24 21:47:00,2177-03-25 09:57:00,224385,59.5,2177-03-29 13:11:00,227194.0,59.5,Planned Extubation,4 days 03:14:00,2177-03-29 14:15:00,2177-03-29 14:15:00,1,1


In [113]:
#intubation에서 extubation까지 수행한 시간을 계산하겠음 (ventilator 적용시간 보고자함)
intubation_extubation3['intubationtoextubationtime'] = intubation_extubation3['extubationtime'] - intubation_extubation3['intubationtime']

In [114]:
intubation_extubation3.sort_values(by = ['subject_id','hadm_id','admittime','intubationtime','extubationtime'], inplace=True)

In [115]:
intubation_extubation3.head(50) #intseq 좀 이상함. 다시.

,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime,intcount,intseq
0,10001884,26184834,2131-01-07 20:39:00,2131-01-11 04:30:00,224385,65.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,1 days 13:10:00,2131-01-20 05:15:00,2131-01-20 05:15:00,1,1
1,10003400,23559586,2137-08-04 00:07:00,2137-08-17 21:21:00,224385,90.5,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,3 days 18:19:00,2137-09-02 17:05:00,2137-09-02 17:05:00,1,1
2,10004401,27939719,2144-04-11 03:31:00,2144-04-11 05:03:00,224385,120.0,2144-04-13 17:31:00,NaN,NaN,dischtime,2 days 12:28:00,2144-04-13 17:31:00,NaT,1,1
3,10004401,29988601,2144-01-23 07:58:00,2144-01-27 19:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2 days 17:30:00,2144-02-06 11:45:00,NaT,2,1
4,10004401,29988601,2144-01-23 07:58:00,2144-01-30 13:00:00,224385,76.0,2144-02-06 11:45:00,227194.0,76.0,Planned Extubation,6 days 22:45:00,2144-02-06 11:45:00,NaT,2,2
5,10005817,28661809,2135-01-03 21:54:00,2135-01-06 08:33:00,224385,88.8,2135-01-19 14:00:00,227194.0,88.8,Planned Extubation,13 days 05:27:00,2135-01-19 18:36:00,2135-01-19 18:36:00,1,1
6,10007818,22987108,2146-06-10 16:37:00,2146-06-22 12:49:00,224385,86.2,2146-07-12 00:00:00,NaN,NaN,dischtime,19 days 11:11:00,2146-07-12 00:00:00,2146-07-12 20:50:00,1,1
7,10010058,26359957,2147-11-18 02:34:00,2147-11-18 02:35:00,224385,60.0,2147-11-19 04:00:00,NaN,NaN,dischtime,1 days 01:25:00,2147-11-19 04:00:00,2147-11-19 04:00:00,1,1
8,10011365,26712576,2166-02-11 14:09:00,2166-02-14 17:59:00,224385,46.3,2166-02-20 00:20:00,227194.0,46.3,Planned Extubation,5 days 06:21:00,2166-02-20 00:20:00,2166-02-20 00:20:00,1,1
9,10015931,22130791,2177-03-24 21:47:00,2177-03-25 09:57:00,224385,59.5,2177-03-29 13:11:00,227194.0,59.5,Planned Extubation,4 days 03:14:00,2177-03-29 14:15:00,2177-03-29 14:15:00,1,1


In [116]:
intubation_extubation3.sort_values(by = ['subject_id','hadm_id','admittime','intubationtime','extubationtime'], inplace=True)
intubation_extubation3.reset_index(drop=True, inplace=True)

In [117]:
#hadm_id 별로 intubationtime 이 총 몇개 있는지 확인하고 싶음(각 입원에서 몇번 intubation을 했는지 확인하고 싶음) 이걸 intcount 라는 컬럼에 넣어주면 좋겠음
intubation_extubation3['intcount'] = intubation_extubation3.groupby('hadm_id')['intubationtime'].transform('count')
#intseq 라는 컬럼에 같은 hadm_id 에서 intubationtime 이 빠른 순서대로 1, 2 ... 와 같이 넘버링하고 싶어
intubation_extubation3['intseq'] = intubation_extubation3.groupby('hadm_id').cumcount() + 1

In [118]:
intubation_extubation3.head(20)

,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime,intcount,intseq
0,10001884,26184834,2131-01-07 20:39:00,2131-01-11 04:30:00,224385,65.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,1 days 13:10:00,2131-01-20 05:15:00,2131-01-20 05:15:00,1,1
1,10003400,23559586,2137-08-04 00:07:00,2137-08-17 21:21:00,224385,90.5,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,3 days 18:19:00,2137-09-02 17:05:00,2137-09-02 17:05:00,1,1
2,10004401,27939719,2144-04-11 03:31:00,2144-04-11 05:03:00,224385,120.0,2144-04-13 17:31:00,NaN,NaN,dischtime,2 days 12:28:00,2144-04-13 17:31:00,NaT,1,1
3,10004401,29988601,2144-01-23 07:58:00,2144-01-27 19:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2 days 17:30:00,2144-02-06 11:45:00,NaT,2,1
4,10004401,29988601,2144-01-23 07:58:00,2144-01-30 13:00:00,224385,76.0,2144-02-06 11:45:00,227194.0,76.0,Planned Extubation,6 days 22:45:00,2144-02-06 11:45:00,NaT,2,2
5,10005817,28661809,2135-01-03 21:54:00,2135-01-06 08:33:00,224385,88.8,2135-01-19 14:00:00,227194.0,88.8,Planned Extubation,13 days 05:27:00,2135-01-19 18:36:00,2135-01-19 18:36:00,1,1
6,10007818,22987108,2146-06-10 16:37:00,2146-06-22 12:49:00,224385,86.2,2146-07-12 00:00:00,NaN,NaN,dischtime,19 days 11:11:00,2146-07-12 00:00:00,2146-07-12 20:50:00,1,1
7,10010058,26359957,2147-11-18 02:34:00,2147-11-18 02:35:00,224385,60.0,2147-11-19 04:00:00,NaN,NaN,dischtime,1 days 01:25:00,2147-11-19 04:00:00,2147-11-19 04:00:00,1,1
8,10011365,26712576,2166-02-11 14:09:00,2166-02-14 17:59:00,224385,46.3,2166-02-20 00:20:00,227194.0,46.3,Planned Extubation,5 days 06:21:00,2166-02-20 00:20:00,2166-02-20 00:20:00,1,1
9,10015931,22130791,2177-03-24 21:47:00,2177-03-25 09:57:00,224385,59.5,2177-03-29 13:11:00,227194.0,59.5,Planned Extubation,4 days 03:14:00,2177-03-29 14:15:00,2177-03-29 14:15:00,1,1


In [119]:
#intseq 가 1이고, intubationtoextubationtime 이 음수면, admittime을 intubationtime으로 대체함
def replace_intubationtime(row):
    if row['intseq'] == 1 and row['intubationtoextubationtime'].days < 0:
        return row['admittime']
    return row['intubationtime']

# apply 함수를 사용하여 'intubationtime' 컬럼을 업데이트합니다.
intubation_extubation3['intubationtime'] = intubation_extubation3.apply(replace_intubationtime, axis=1)


In [120]:
#intubation에서 extubation까지 수행한 시간을 계산하겠음 (ventilator 적용시간 보고자함)
intubation_extubation3['intubationtoextubationtime'] = intubation_extubation3['extubationtime'] - intubation_extubation3['intubationtime']

In [121]:
intubation_extubation4 = intubation_extubation3

In [122]:
intubation_extubation4.to_csv('../outputs/intubation_extubation4.csv', index=False)

In [123]:
intubation_extubation4.head(30)

,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime,intcount,intseq
0,10001884,26184834,2131-01-07 20:39:00,2131-01-11 04:30:00,224385,65.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,1 days 13:10:00,2131-01-20 05:15:00,2131-01-20 05:15:00,1,1
1,10003400,23559586,2137-08-04 00:07:00,2137-08-17 21:21:00,224385,90.5,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,3 days 18:19:00,2137-09-02 17:05:00,2137-09-02 17:05:00,1,1
2,10004401,27939719,2144-04-11 03:31:00,2144-04-11 05:03:00,224385,120.0,2144-04-13 17:31:00,NaN,NaN,dischtime,2 days 12:28:00,2144-04-13 17:31:00,NaT,1,1
3,10004401,29988601,2144-01-23 07:58:00,2144-01-27 19:00:00,224385,76.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2 days 17:30:00,2144-02-06 11:45:00,NaT,2,1
4,10004401,29988601,2144-01-23 07:58:00,2144-01-30 13:00:00,224385,76.0,2144-02-06 11:45:00,227194.0,76.0,Planned Extubation,6 days 22:45:00,2144-02-06 11:45:00,NaT,2,2
5,10005817,28661809,2135-01-03 21:54:00,2135-01-06 08:33:00,224385,88.8,2135-01-19 14:00:00,227194.0,88.8,Planned Extubation,13 days 05:27:00,2135-01-19 18:36:00,2135-01-19 18:36:00,1,1
6,10007818,22987108,2146-06-10 16:37:00,2146-06-22 12:49:00,224385,86.2,2146-07-12 00:00:00,NaN,NaN,dischtime,19 days 11:11:00,2146-07-12 00:00:00,2146-07-12 20:50:00,1,1
7,10010058,26359957,2147-11-18 02:34:00,2147-11-18 02:35:00,224385,60.0,2147-11-19 04:00:00,NaN,NaN,dischtime,1 days 01:25:00,2147-11-19 04:00:00,2147-11-19 04:00:00,1,1
8,10011365,26712576,2166-02-11 14:09:00,2166-02-14 17:59:00,224385,46.3,2166-02-20 00:20:00,227194.0,46.3,Planned Extubation,5 days 06:21:00,2166-02-20 00:20:00,2166-02-20 00:20:00,1,1
9,10015931,22130791,2177-03-24 21:47:00,2177-03-25 09:57:00,224385,59.5,2177-03-29 13:11:00,227194.0,59.5,Planned Extubation,4 days 03:14:00,2177-03-29 14:15:00,2177-03-29 14:15:00,1,1


In [124]:
intubation_extubation4[intubation_extubation4['intubationtoextubationtime'] < pd.Timedelta(0)]
#372개를 확인해야 할듯

,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime,intcount,intseq
69,10098215,22394571,2118-05-03 23:07:00,2118-06-18 01:48:00,224385,48.7,2118-06-08 11:55:00,227194.0,55.4,Planned Extubation,-10 days +10:07:00,2118-07-16 12:15:00,NaT,12,7
72,10098215,22394571,2118-05-03 23:07:00,2118-06-21 08:28:00,224385,48.7,2118-06-08 11:55:00,227194.0,55.4,Planned Extubation,-13 days +03:27:00,2118-07-16 12:15:00,NaT,12,10
73,10098215,22394571,2118-05-03 23:07:00,2118-06-21 08:28:00,224385,48.7,2118-06-21 03:30:00,225468.0,48.7,Unplanned Extuabtion (patient-initiated),-1 days +19:02:00,2118-07-16 12:15:00,NaT,12,11
101,10125047,26680668,2180-01-18 23:15:00,2180-01-24 15:20:00,224385,65.6,2180-01-20 08:05:00,227194.0,65.6,Planned Extubation,-5 days +16:45:00,2180-02-01 13:30:00,NaT,4,3
123,10150335,24113307,2144-09-09 20:15:00,2144-09-17 01:00:00,224385,66.3,2144-09-12 18:50:00,227194.0,66.3,Planned Extubation,-5 days +17:50:00,2144-09-17 18:30:00,2144-09-17 18:30:00,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8869,19913577,20355379,2114-05-01 20:02:00,2114-05-21 07:53:00,224385,85.0,2114-05-17 09:00:00,227194.0,85.0,Planned Extubation,-4 days +01:07:00,2114-06-14 00:01:00,2114-06-14 20:25:00,9,5
8871,19913577,20355379,2114-05-01 20:02:00,2114-05-27 19:51:00,224385,85.0,2114-05-11 16:00:00,227194.0,85.0,Planned Extubation,-17 days +20:09:00,2114-06-14 00:01:00,2114-06-14 20:25:00,9,7
8872,19913577,20355379,2114-05-01 20:02:00,2114-05-27 19:51:00,224385,85.0,2114-05-17 09:00:00,227194.0,85.0,Planned Extubation,-11 days +13:09:00,2114-06-14 00:01:00,2114-06-14 20:25:00,9,8
8915,19969918,29544887,2186-01-15 23:00:00,2186-01-27 18:30:00,224385,68.0,2186-01-18 09:56:00,227194.0,68.0,Planned Extubation,-10 days +15:26:00,2186-02-07 17:04:00,NaT,2,2


In [125]:
error_hadm_ids = intubation_extubation4[intubation_extubation4['intubationtoextubationtime'] < pd.Timedelta(0)]['hadm_id'].unique()
error_rows = intubation_extubation4[intubation_extubation4['hadm_id'].isin(error_hadm_ids)]
display(error_rows)

,subject_id,hadm_id,admittime,intubationtime,int_itemid,int_weight,extubationtime,ext_itemid,ext_weight,extubationcause,intubationtoextubationtime,dischtime,deathtime,intcount,intseq
63,10098215,22394571,2118-05-03 23:07:00,2118-06-03 20:50:00,224385,55.4,2118-06-08 11:55:00,227194.0,55.4,Planned Extubation,4 days 15:05:00,2118-07-16 12:15:00,NaT,12,1
64,10098215,22394571,2118-05-03 23:07:00,2118-06-03 20:50:00,224385,55.4,2118-06-21 03:30:00,225468.0,48.7,Unplanned Extuabtion (patient-initiated),17 days 06:40:00,2118-07-16 12:15:00,NaT,12,2
65,10098215,22394571,2118-05-03 23:07:00,2118-06-03 20:50:00,224385,55.4,2118-06-27 12:45:00,227194.0,48.7,Planned Extubation,23 days 15:55:00,2118-07-16 12:15:00,NaT,12,3
66,10098215,22394571,2118-05-03 23:07:00,2118-06-07 01:40:00,224385,55.4,2118-06-08 11:55:00,227194.0,55.4,Planned Extubation,1 days 10:15:00,2118-07-16 12:15:00,NaT,12,4
67,10098215,22394571,2118-05-03 23:07:00,2118-06-07 01:40:00,224385,55.4,2118-06-21 03:30:00,225468.0,48.7,Unplanned Extuabtion (patient-initiated),14 days 01:50:00,2118-07-16 12:15:00,NaT,12,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8940,19997367,20617667,2126-04-20 07:15:00,2126-04-26 20:14:00,224385,59.0,2126-05-02 16:30:00,227194.0,59.0,Planned Extubation,5 days 20:16:00,2126-05-19 14:15:00,NaT,6,2
8941,19997367,20617667,2126-04-20 07:15:00,2126-04-26 20:14:00,224385,59.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,11 days 21:15:00,2126-05-19 14:15:00,NaT,6,3
8942,19997367,20617667,2126-04-20 07:15:00,2126-05-05 18:20:00,224385,59.0,2126-05-02 16:30:00,227194.0,59.0,Planned Extubation,-4 days +22:10:00,2126-05-19 14:15:00,NaT,6,4
8943,19997367,20617667,2126-04-20 07:15:00,2126-05-05 18:20:00,224385,59.0,2126-05-08 17:29:00,227194.0,59.0,Planned Extubation,2 days 23:09:00,2126-05-19 14:15:00,NaT,6,5


In [126]:
tunnel.close()